In [57]:
from pycuda import gpuarray
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
ker = SourceModule("""
#define _x  ( threadIdx.x + blockIdx.x * blockDim.x )
#define _y  ( threadIdx.y + blockIdx.y * blockDim.y )
#define _z  ( threadIdx.z + blockIdx.z * blockDim.z )
#define _width  ( blockDim.x * gridDim.x )
#define _height ( blockDim.y * gridDim.y  )
#define _depth  ( blockDim.z * gridDim.z  )
#define _xm(x)  ( (x + _width) % _width )
#define _ym(y)  ( (y + _height) % _height )
#define _zm(z)  ( (z + _depth) % _depth )
#define _index(x,y,z)  ( _zm(z)  + _depth * (_ym(y) + _xm(x) * _height) )

__global__ void poisson_out_of_place(float* V_new, float* V, int imax, int jmax, int kmax) {
    int x = _x, y = _y, z = _z;
    float r = 0;
    if (x >= 1  && y >= 1 && z >= 1 && x < imax+1 && y < jmax+1 && z < kmax-1) {
            r = V[_index(x+1,y,z)]
                     +V[_index(x-1,y,z)]
                     +V[_index(x,y+1,z)]
                     +V[_index(x,y-1,z)]
                     +V[_index(x,y,z+1)]
                     +V[_index(x,y,z-1)];
            r = r/6.0-V[_index(x,y,z)]/6.0;
            V_new[_index(x,y,z)]=r;                   
    }
}
__global__ void poisson_in_place(float* V, int iters, int imax, int jmax, int kmax, int oddEven) {
    //int x = _x, y = _y, z = _z;
    for (int kk=0; kk<iters; kk++) {
        for (int x = _x; x < imax+1; x += _width) {
            for (int y = _y; y < jmax+1; y += _height) {
                for (int z = _z; z < kmax-1; z += _depth) {
                    float r = 0;
                    if (x >= 1  && y >= 1 && z >= 1 ) {
                        if ((x+y+z)%2==oddEven) {
                            r = V[_index(x+1,y,z)]
                                     +V[_index(x-1,y,z)]
                                     +V[_index(x,y+1,z)]
                                     +V[_index(x,y-1,z)]
                                     +V[_index(x,y,z+1)]
                                     +V[_index(x,y,z-1)];
                            r = r/6.0-V[_index(x,y,z)]/6.0;
                        }
                        V[_index(x,y,z)]=r;                   
                    }
                }
            }
        }
    }
}
__global__ void test(float* V) {
    int x = _x, y = _y, z = _z;
    V[_index(x,y,z)] = _index(x,y,z);
}
""")


poisson_ker_out_of_place = ker.get_function("poisson_out_of_place")
poisson_ker_in_place = ker.get_function("poisson_in_place")
test_ker    = ker.get_function("test")

In [59]:
n1 = n2 = n3 = np.int32(16)
v = np.zeros((n1, n2, n3)).astype(np.float32)
v_gpu = gpuarray.to_gpu(v)
test_ker(v_gpu, n1, n2, n3,  grid=(int(n1)//16,int(n2)//16,int(n3)//4), block=(16,16,4))
v = v_gpu.get()
for i in range(n1*n2*n3):
    unravelled_index = np.unravel_index(i, v.shape)
    if i != int(v[unravelled_index]):
        print(i, unravelled_index, v[unravelled_index])
        break

In [60]:
def python_poisson_out_of_place(V, iters, imax, jmax, kmax):
    R = V.copy()
    for kk in range(iters):
        for x in range(1, imax+1):
            for y in range(1, jmax+1):
                for z in range(1, kmax-1):
                    r = V[x+1,y,z]\
                             +V[x-1,y,z]\
                             +V[x,y+1,z]\
                             +V[x,y-1,z]\
                             +V[x,y,z+1]\
                             +V[x,y,z-1]
                    r = r/6.0-V[x,y,z]/6.0;
                    R[x,y,z]=r
        V[:] = R[:]
    return R
def python_poisson_in_place(V, iters, imax, jmax, kmax):
    for kk in range(iters):
        for x in range(1, imax+1):
            for y in range(1, jmax+1):
                for z in range(1, kmax-1):
                    r = V[x+1,y,z]\
                             +V[x-1,y,z]\
                             +V[x,y+1,z]\
                             +V[x,y-1,z]\
                             +V[x,y,z+1]\
                             +V[x,y,z-1]
                    r = r/6.0-V[x,y,z]/6.0;
                    V[x,y,z]=r

In [61]:
imax = jmax = kmax = 14
V_orig = np.random.rand(imax+2, jmax+2, kmax+2).astype(np.float32) * 10
V_orig[0] = 0
V_orig[:,0,:] = 0
V_orig[:,:,0] = 0
V_orig[-1] = 0
V_orig[:,-1,:] = 0
V_orig[:,:,-1] = 0
g = np.random.rand(imax+2, jmax+2, kmax+2).astype(np.float32) * 10
n_iters = 1

# out of place check

In [62]:
V = V_orig.copy()
V = python_poisson_out_of_place(V, n_iters, imax, jmax, kmax)

In [63]:
V_gpu  = gpuarray.to_gpu(V_orig.copy())
V2_gpu = gpuarray.to_gpu(V_orig.copy())
g_gpu  = gpuarray.to_gpu(g)
for _ in range(n_iters):
    poisson_ker_out_of_place(V2_gpu, V_gpu, np.int32(imax), np.int32(jmax), np.int32(kmax), grid=(2,2,2), block=(8, 8, 8))
    tmp = V_gpu
    V_gpu = V2_gpu
    V2_gpu = tmp
    #V_gpu[:] = V2_gpu[:]

In [64]:
max_index = np.argmax(np.abs(V_gpu.get() - V))
unravelled_index = np.unravel_index(max_index, V.shape)
unravelled_index

(0, 0, 0)

In [65]:
print(V[unravelled_index])
print(V_gpu.get()[unravelled_index])
np.allclose(V_gpu.get(), V)

0.0
0.0


True

# in place check

In [15]:
V = V_orig.copy()
python_poisson_in_place(V, n_iters, imax, jmax, kmax)

In [16]:
V_gpu  = gpuarray.to_gpu(V_orig.copy())
g_gpu  = gpuarray.to_gpu(g)
for _ in range(n_iters):
    poisson_ker_in_place(V_gpu, np.int32(1), np.int32(imax), np.int32(jmax), np.int32(kmax), np.int32(1), grid=(1,1,32), block=(32, 32, 1))
    poisson_ker_in_place(V_gpu, np.int32(1), np.int32(imax), np.int32(jmax), np.int32(kmax), np.int32(0), grid=(1,1,32), block=(32, 32, 1))

In [17]:
max_index = np.argmax(np.abs(V_gpu.get() - V))
unravelled_index = np.unravel_index(max_index, V.shape)

In [18]:
print(V[unravelled_index])
print(V_gpu.get()[unravelled_index])
np.allclose(V_gpu.get(), V)

2.974674
0.0


False

In [55]:
unravelled_index

(10, 29, 3)

In [173]:
V_gpu.get()[unravelled_index]

4.425444

In [175]:
V[unravelled_index]

1.9709917

In [181]:
len(np.where(np.abs(V_gpu.get() - V) > 1)[0])

739